In [2]:
import socket
import threading
import time
import os
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

# -----------------------------
# CONFIGURATION
# -----------------------------
FILE_PATH = r"C:\wireshark\synthetic_data_1mb.bin"
HOST = '127.0.0.1'
PORT = 5003
BUFFER_SIZE = 4096

# Shared variable for classical timing
classical_time_result = {"server_time": 0.0, "client_time": 0.0}

# -----------------------------
# Classical TCP Transmission
# -----------------------------
def tcp_server():
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind((HOST, PORT))
    server_socket.listen(1)
    conn, addr = server_socket.accept()

    start = time.time()
    with open("received_classical.bin", "wb") as f:
        while True:
            data = conn.recv(BUFFER_SIZE)
            if not data:
                break
            f.write(data)
    end = time.time()

    classical_time_result["server_time"] = end - start
    print(f"[SERVER] Classical transmission time: {classical_time_result['server_time']:.6f} sec")

    conn.close()
    server_socket.close()


def tcp_client():
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket.connect((HOST, PORT))
    start = time.time()
    with open(FILE_PATH, "rb") as f:
        while True:
            bytes_read = f.read(BUFFER_SIZE)
            if not bytes_read:
                break
            client_socket.sendall(bytes_read)
    end = time.time()
    classical_time_result["client_time"] = end - start
    print(f"[CLIENT] Classical transmission time: {classical_time_result['client_time']:.6f} sec")
    client_socket.close()


# Run classical transmission
server_thread = threading.Thread(target=tcp_server)
server_thread.start()
time.sleep(1)
tcp_client()
server_thread.join()

# Take average of server and client timing
classical_time = (classical_time_result["server_time"] + classical_time_result["client_time"]) / 2

# -----------------------------
# Quantum Transmission Simulation
# -----------------------------
def quantum_transmission_simulation():
    start = time.time()

    simulator = AerSimulator()
    qc = QuantumCircuit(3, 3)

    # Superdense + teleportation combined simulation
    qc.h(1)
    qc.cx(1, 0)
    qc.x(1)
    qc.barrier()

    qc.cx(2, 1)
    qc.h(2)
    qc.measure([2, 1], [0, 1])
    qc.barrier()
    qc.cx(1, 0)
    qc.cz(2, 0)

    t_qc = transpile(qc, simulator)
    simulator.run(t_qc).result()

    end = time.time()
    quantum_time = end - start
    print(f"[QUANTUM] Transmission time (simulation): {quantum_time:.6f} sec")
    return quantum_time


quantum_time = quantum_transmission_simulation()

# -----------------------------
# Comparison
# -----------------------------
print("\n===== Transmission Time Comparison =====")
print(f"Classical (TCP):  {classical_time:.6f} sec (avg of client/server)")
print(f"Quantum (Simulated): {quantum_time:.6f} sec")

if quantum_time < classical_time:
    print("  Quantum transmission appears faster (simulation).")
else:
    print("  Classical TCP was faster this run.")


[CLIENT] Classical transmission time: 0.186324 sec
[SERVER] Classical transmission time: 0.187174 sec
[QUANTUM] Transmission time (simulation): 0.482485 sec

===== Transmission Time Comparison =====
Classical (TCP):  0.186749 sec (avg of client/server)
Quantum (Simulated): 0.482485 sec
🕓 Classical TCP was faster this run.


In [4]:
from qiskit import QuantumCircuit
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
import time

# Connect to IBM Quantum
service = QiskitRuntimeService(
    channel="ibm_cloud",
    token="w8F2BT6E7Ls-qpVbNVQekTkIMC1-cOIRagh8XtGDNBTb",
    instance="open-instance"
)
print("✅ Connected to IBM Quantum Runtime")

# ----------------------------
# Pick a simulator backend OBJECT (not string!)
# ----------------------------
backends = service.backends(simulator=True, operational=True)
if backends:
    backend_obj = backends[0]  # this is a Backend object
else:
    raise RuntimeError("No operational simulator backend found")

print("Using backend:", backend_obj.name)

# ----------------------------
# Example Dense Coding Circuit
# ----------------------------
def dense_coding(bit1, bit2):
    qc = QuantumCircuit(2, 2)
    qc.h(0)
    qc.cx(0,1)
    if bit2: qc.x(0)
    if bit1: qc.z(0)
    qc.cx(0,1)
    qc.h(0)
    qc.measure([0,1],[0,1])
    return qc

# ----------------------------
# Run quantum simulation
# ----------------------------
def run_quantum_simulation(qc, backend_obj):
    start_time = time.time()
    sampler = Sampler(mode=backend_obj)  # PASS backend OBJECT, not name
    job = sampler.run(qc, shots=1024)
    result = job.result()
    elapsed = time.time() - start_time
    return result.quasi_dists, elapsed

# ----------------------------
# Test
# ----------------------------
qc = dense_coding(1, 0)
result, elapsed_time = run_quantum_simulation(qc, backend_obj)
print("Result:", result)
print("Time:", elapsed_time)


qiskit_runtime_service._discover_account:WARNING:2025-11-10 20:45:18,462: Loading account with the given token. A saved account will not be used.


✅ Connected to IBM Quantum Runtime


RuntimeError: No operational simulator backend found

In [1]:
!pip install qiskit-ibm-provider --upgrade


In [9]:
from qiskit import QuantumCircuit
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler, Session

# Connect to IBM Quantum Runtime
service = QiskitRuntimeService(
    channel="ibm_cloud",
    token="w8F2BT6E7Ls-qpVbNVQekTkIMC1-cOIRagh8XtGDNBTb",
    instance="open-instance"
)
print("✅ Connected to IBM Quantum Runtime")

# ----------------------------
# Pick a simulator backend
# ----------------------------
# Try to get a runtime simulator
backends = service.backends(simulator=True, operational=True)

if backends:
    backend_obj = backends[0]  # pick first available
else:
    # fallback: use classical runtime simulator
    from qiskit_ibm_runtime import QiskitRuntimeLocalService
    backend_obj = QiskitRuntimeLocalService()  # local simulator
    print("⚠️ Using local simulator as fallback")

print("Using backend:", getattr(backend_obj, "name", "local_simulator"))

# ----------------------------
# Define circuits
# ----------------------------
def dense_coding(bit1, bit2):
    qc = QuantumCircuit(2, 2)
    qc.h(0)
    qc.cx(0, 1)
    if bit2: qc.x(0)
    if bit1: qc.z(0)
    qc.cx(0, 1)
    qc.h(0)
    qc.measure([0,1],[0,1])
    return qc

def quantum_teleportation(bit):
    qc = QuantumCircuit(3, 2)
    if bit: qc.x(0)
    qc.h(1)
    qc.cx(1,2)
    qc.cx(0,1)
    qc.h(0)
    qc.measure([0,1],[0,1])
    qc.cx(1,2)
    qc.cz(0,2)
    qc.measure(2,0)
    return qc

# ----------------------------
# Run simulation
# ----------------------------
def run_quantum_simulation(qc, backend_obj):
    sampler = Sampler(mode=backend_obj)  # pass backend object
    job = sampler.run(qc, shots=1024)
    result = job.result()
    return result.quasi_dists

# ----------------------------
# Example run
# ----------------------------
dense_qc = dense_coding(1,0)
teleport_qc = quantum_teleportation(1)

dense_result = run_quantum_simulation(dense_qc, backend_obj)
teleport_result = run_quantum_simulation(teleport_qc, backend_obj)

print("Dense Coding Result:", dense_result)
print("Quantum Teleportation Result:", teleport_result)


qiskit_runtime_service._discover_account:WARNING:2025-11-10 21:37:28,127: Loading account with the given token. A saved account will not be used.


InvalidAccountError: 'Unable to retrieve instances. Please check that you are using a valid API token.'

In [8]:
pip install --upgrade qiskit qiskit-ibm-runtime


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: qiskit in c:\users\mahno\anaconda3\lib\site-packages (2.2.3)

